In [1]:
import numpy as np, pandas as pd
import subprocess, json
from Stats import *

from Circuit_Properties import *
from Stats import *
from Functions import *
from HexProperties import *
from rstools import RSoftUserFunction, RSoftCircuit
'''
Set RSoft parameters, dump them in a json file and generate the RSoft circuit
'''

name = "MCF_Test"
sym = {}

# set fibre parameters to be used in template file
length = 1000
Corediam = 8.2
Claddiam = 125
Core_sep = 60
taper_ratio = 1
core_num = 1 # this must be an odd number!!
Core_delta = 0.01
background_index = 1.456
delta = 0.012

# these determine the multiprocessing parameters
num_paras = 100
batch_number = 6

# simulation stuff
Dx = 0.1
Dy = Dx
Dz = 0.2
Phase = 0
boundary_gap_x = 10
boundary_gap_y = boundary_gap_x
boundary_gap_z = 0
bpm_pathway = 1
bpm_pathway_monitor = bpm_pathway
sym_tool = Sim_tool.BP
width = 5
height = width
H = height
grid_uniform = 0
eim = 0
polarization = 0
free_space_wavelength = 1
k0 = 2*np.pi / free_space_wavelength
slice_display_mode = 'DISPLAY_CONTOURMAPXZ'
slice_position_z = 100

# launch properties
launch_tilt = 1
launch_port = 1
launch_align_file = 1
launch_mode = 0
launch_mode_radial = 1
launch_normalization = 1
grid_size = 'Dx'
grid_size_y = 'Dy'
step_size = 'Dz'

structure = Struct_type.FIBRE

#################################################################################
'''
Json Derulo
'''
# dump variable parameters in json file
with open("variable_paras.json", "w") as g:
        json.dump({"Name": name,
                   "Length": length,
                   "Corediam": Corediam,
                   "Claddiam": Claddiam,
                   "Core_sep": Core_sep,
                   "taper_ratio": taper_ratio,
                   "core_num": core_num,
                   "Core_delta": Core_delta,
                   "background_index": background_index,
                   "delta": delta
        }, g)

# dump fixed parameters in json file
with open("fibre_prop.json", "w") as f:
        json.dump({"Dx": Dx,
                   "Dy": Dy,
                   "Dz": Dz,
                   "Phase": Phase,
                   "boundary_gap_x": boundary_gap_x,
                   "boundary_gap_y": boundary_gap_y,
                   "boundary_gap_z": boundary_gap_z,
                   "bpm_pathway": bpm_pathway,
                   "bpm_pathway_monitor": bpm_pathway_monitor,
                   "sym_tool": sym_tool,
                   "width": width,
                   "height": height,
                   "H": height,
                   "grid_uniform": grid_uniform,
                   "eim": eim,
                   "polarization": polarization,
                   "free_space_wavelength": free_space_wavelength,
                   "k0": k0,
                   "slice_display_mode": slice_display_mode,
                   "slice_position_z": slice_position_z,
                   "launch_tilt": launch_tilt,
                   "launch_port": launch_port,
                   "launch_align_file": launch_align_file,
                   "launch_mode": launch_mode,
                   "launch_mode_radial": launch_mode_radial,
                   "launch_normalization": launch_normalization,
                   "grid_size": grid_size,
                   "grid_size_y": grid_size_y,
                   "step_size": step_size,
                   "structure": structure,
                   "num_paras": num_paras,
                   "batch_number": batch_number
                   }, f)

# properties of the structure (units of um)
with open("variable_paras.json","r") as g:
    param_v = json.load(g)
with open("fibre_prop.json", "r") as f:
    params = json.load(f)

# initialise prior space
para_space = {
    "Corediam": (1.0, 50.0),
    "Length": (10.0, 1000.0),
    "Core_index": (1.0, 2.0) # note: if upper limit > 5 simulation fails. Make the lower prior be 1.
}

# dump prior space
with open("prior_space.json", "w") as write:
    json.dump(para_space, write)
#################################################################################

# assign parameters to RSoft circuit
for key_v in param_v:
    sym[key_v] = param_v[key_v]
for key in params:
    sym[key] = params[key]

# creating the design file, load the settings and add symbols
c = RSoftCircuit()
for key in sym:
    c.set_symbol(key,sym[key])

In [2]:
''' 
Generating the positional coordinates for each fibre 
'''

if core_num % 2 == 0:
    raise ValueError(f"The number of cores must be odd to perfectly fit inside the hex grid. Received:{core_num}")

# checking to see if the cores fully populate the hex grid
row_numbers = [number_rows(core_num)]
'''
TO DO: add options for the distribution of cores, make this a function that takes a string ("Hex", "Pent", "Circle" etc)
'''
for idx, row_num in enumerate(row_numbers):
    hcoord, vcoord = generate_hex_grid(row_num, sym["Core_sep"])

In [3]:
''' 
Generating the segments and assigning the positional coordinates found in 
the previous sections 
'''

# Add the outer cladding segment
cladding_beg_dims = (sym['Claddiam'] / sym['taper_ratio'], sym['Claddiam'] / sym['taper_ratio'])
cladding_end_dims = (sym['Claddiam'] , sym['Claddiam'])

# Add the individual core segments
core_beg_dims = (sym['Corediam']/ sym['taper_ratio'],sym['Corediam']/ sym['taper_ratio'])
core_end_dims = (sym['Corediam'],sym['Corediam'])

if core_num == 1: 
    cladding = c.add_segment(
        position=(0, 0, 0),
        offset=(0, 0, 'Length'),
        dimensions=core_beg_dims,
        dimensions_end=core_end_dims
    )
    cladding.set_name("MMF Cladding")
else:
    cladding = c.add_segment(
        position=(0, 0, 0),
        offset=(0, 0, 'Length'),
        dimensions=core_beg_dims,
        dimensions_end=core_end_dims
    )
    cladding.set_name("MMF Cladding")
    
    core_name = [f"core_{n+1:02}" for n in range(sym['core_num'])]

    for j, (x, y) in enumerate(zip(hcoord, vcoord)):
        core = c.add_segment(
                    position=(x/sym['taper_ratio'],y/sym['taper_ratio'],0), 
                    offset=(x,y,'Length'), 
                    dimensions = core_beg_dims, 
                    dimensions_end = core_end_dims
                    )
        core.set_name(core_name[j])

# write to file
c.write(f"{name}.ind")

'''
This is the hacking part which adds all of the pathways, then the monitors and then the 
launch fields. They are done individually to account for formatting issues that may exist
within RSoft's .ind files.
'''
AddHack(name, int(core_num), Corediam, Core_delta)

In [4]:
'''
Multiprocessing must be run outside of a Jupyter cell or else it will silently fail/infinitely loop on the first batch
'''
subprocess.run(["python", "Multiprocessing.py"], check=True)

CompletedProcess(args=['python', 'Multiprocessing.py'], returncode=0)

In [5]:
# Read the multiprocessing csv log into a DataFrame
df = pd.read_csv("best_params_log.csv", header=None)

param_columns = [prior_name for prior_name, (_,_) in para_space.items()] + ["Throughput"]

print("Best ftting values:")
for col in df.columns[1:]:
    print(f"{param_columns[col-1]}: {df[col].iloc[-1]}")
    


Best ftting values:
Corediam: 3.217335220428569
Length: 24.01163962140473
Core_index: 1.4706654384294064
Throughput: 0.9999999512396698
